In [1]:
# connected GPU information
!nvidia-smi

Wed Feb 10 16:04:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
# install huggingface transformers
!pip -q install transformers

     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 3.2MB 63.3MB/s 
     |████████████████████████████████| 890kB 65.3MB/s 


In [4]:
import os

import tensorflow as tf
import transformers

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random as rnd
rnd.seed(100)

In [5]:
import re
import string

import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import warnings
warnings.filterwarnings('ignore')

<a name='1'></a>
#Importing the Data

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Duplicate Question Detection/quora_questions.csv")
#data.fillna("none value", inplace=True) # replace nan value to none
# drop the rows with null value
data.dropna(axis=0, inplace=True)
N=len(data)
print('Number of question pairs: ', N)
data.head()

Number of question pairs:  404348


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [31]:
# take random 20% data
total_data = 404348
n = 80800
skip = sorted(rnd.sample(range(1, total_data), total_data- n))
data = pd.read_csv("/content/drive/My Drive/Duplicate Question Detection/quora_questions.csv", skiprows=skip)
#data.fillna("none value", inplace=True) # replace nan value to none
# drop the rows with null value
data.dropna(axis=0, inplace=True)
N=len(data)
print('Number of question pairs: ', N)
data.head()

Number of question pairs:  80802


,id,qid1,qid2,question1,question2,is_duplicate
0,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
1,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
2,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
3,18,37,38,Why are so many Quora users posting questions ...,Why do people ask Quora questions which can be...,1
4,28,57,58,What is best way to make money online?,What is best way to ask for money online?,0


[18, 58, 98, 22, 90, 50, 93, 44, 55, 64]

In [ ]:
# def cut_to_max(text, max_len):
#     words = text.split()[:max_len]
#     return ' '.join(words)

# # test the func
# # cut_to_max(' abc test wewe dfd ddd', 3)

In [32]:
def cleaning_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    #text = text.lower()

    # Clean the text
    # text = re.sub(r"[^A-Za-z0-9^,!.?\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"What's", "What is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"Can't", "Cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"I'm", "I am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " , ", text)
    text = re.sub(r"\.", " . ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text

In [33]:
data['question1'] = data['question1'].apply(lambda x: cleaning_text(x))
data['question2'] = data['question2'].apply(lambda x: cleaning_text(x))

In [10]:
print("dataset labels Distribution")
print(data.is_duplicate.value_counts())

dataset labels Distribution
0    51216
1    29587
Name: is_duplicate, dtype: int64


We first split the data into a train and test set. The test set will be used later to evaluate our model.

In [66]:
# model selection
from sklearn.model_selection import train_test_split
data_train , data_test = train_test_split(data, train_size=0.8, random_state=0)
# validation_data, _ = train_test_split(data_train, train_size=0.1, random_state=0)
len(data_train), len(data)

(24563, 80802)

In [ ]:
# print("Train dataset Distribution")
# print(data_train.is_duplicate.value_counts())

# print("\n\nTest dataset Distribution")
# print(data_test.is_duplicate.value_counts())

In [67]:
train_Q1 = np.array(data_train['question1'])
train_Q2 = np.array(data_train['question2'])
t_labels = np.array(data_train['is_duplicate'])
# make it one-hot encoding
train_labels = tf.keras.utils.to_categorical(t_labels, num_classes=2)

test_Q1 = np.array(data_test['question1'])
test_Q2 = np.array(data_test['question2'])
te_labels  = np.array(data_test['is_duplicate'])
# make it one-hot encoding
test_labels = tf.keras.utils.to_categorical(te_labels, num_classes=2)

In [53]:
train_Q1.shape, train_Q2.shape, train_labels.shape, test_Q1.shape

((40723,), (40723,), (40723, 2), (16161,))

In [69]:
# data generator
class data_generator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-cased", do_lower_case=False
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded_l = self.tokenizer.batch_encode_plus(
            sentence_pairs[:,0].tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        encoded_r = self.tokenizer.batch_encode_plus(
            sentence_pairs[:,1].tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids_l = np.array(encoded_l["input_ids"], dtype="int32")
        attention_masks_l = np.array(encoded_l["attention_mask"], dtype="int32")

        input_ids_r = np.array(encoded_r["input_ids"], dtype="int32")
        attention_masks_r = np.array(encoded_r["attention_mask"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids_l, attention_masks_l, input_ids_r, attention_masks_r], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [70]:
# make a model for detecting duplicate question

def duplicate_question_detection_model():

    # Encoded token ids from BERT tokenizer.
    input_ids_l = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids_l"
    )

    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks_l = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks_l"
    )

    # Encoded token ids from BERT tokenizer.
    input_ids_r = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids_r"
    )

    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks_r = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks_r"
    )

     # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained('bert-base-cased')
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = True

    outputs_l = bert_model(
        input_ids_l, attention_mask=attention_masks_l
    )

    outputs_r = bert_model(
        input_ids_r, attention_mask=attention_masks_r
    )
    mean_pool_l = tf.keras.layers.GlobalAveragePooling1D()(outputs_l.last_hidden_state)
    mean_pool_r = tf.keras.layers.GlobalAveragePooling1D()(outputs_r.last_hidden_state)

    # mean_pool_l = tf.reduce_mean(sequence_output_l, 1)
    # mean_pool_r = tf.reduce_mean(sequence_output_r, 1)

    subtracted = tf.keras.layers.subtract([mean_pool_l, mean_pool_r])
    diff=tf.abs(subtracted)
    concat = tf.keras.layers.Concatenate()([mean_pool_l, mean_pool_r, diff])
    dropout = tf.keras.layers.Dropout(0.2)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)

    # model object
    model = tf.keras.models.Model(
        inputs=[input_ids_l, attention_masks_l, input_ids_r, attention_masks_r], outputs=output
    )

    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )

    return model


In [71]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    max_length=64
    learning_rate=5e-5
    model = duplicate_question_detection_model()

# print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids_l (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
attention_masks_l (InputLayer)  [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_ids_r (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
attention_masks_r (InputLayer)  [(None, 64)]         0                                            
____________________________________________________________________________________________

In [72]:
# data preparation
batch_size = 32
train_data = data_generator(
    data_train[["question1", "question2"]].values.astype("str"),
    train_labels,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = data_generator(
    data_test[["question1", "question2"]].values.astype("str"),
    test_labels,
    batch_size=batch_size,
    shuffle=False,
)

In [76]:
# change learning rate

tf.keras.backend.set_value(model.optimizer.lr, 1e-5)
print(tf.keras.backend.get_value(model.optimizer.lr))

1e-05


In [77]:
epochs = 1
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

767/767 [==============================] - 421s 549ms/step - loss: 0.1268 - acc: 0.9574 - val_loss: 0.4766 - val_acc: 0.8164


In [ ]:
model.evaluate(valid_data)

505/505 [==============================] - 75s 149ms/step - loss: 0.3767 - acc: 0.8284


[0.37665072083473206, 0.8284034729003906]

In [ ]:
model.save_weights('/content/drive/My Drive/Duplicate Question Detection/Siamese BERT Analysis/siamese_bert_with_stopwords_64_32_03_epoch_2_lr_1_e5.h5', overwrite=True, save_format=None, options=None)

In [ ]:
model.load_weights('/content/drive/My Drive/Duplicate Question Detection/Siamese BERT Analysis/siamese_bert_with_stopwords_64_32_03_epoch_1_lr_5_e5.h5', by_name=False, skip_mismatch=False, options=None)

In [ ]:
preds = model.predict(
    valid_data,
    batch_size=batch_size,
    use_multiprocessing=True
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc, roc_curve, confusion_matrix
y_pred = np.argmax(preds, axis=-1)
n = y_pred.shape[0]
y_true = te_labels[:n]
print('accuracy: ', accuracy_score(y_true, y_pred))
print('F1 score: ', f1_score(y_true, y_pred))
print('Recall score: ', recall_score(y_true, y_pred))
print('Precision score: ', precision_score(y_true, y_pred))

print('confusion matrix: \n', confusion_matrix(y_true, y_pred))

accuracy:  1.0
F1 score:  0.0
Recall score:  0.0
Precision score:  0.0
confusion matrix: 
 [[2]]


In [ ]:
my_data = np.array([['What is the solution for this question?', 'What is the solution to this question?'],
                    ['Who are you?', 'What is your name?']
                    ])
#my_data.shape
custom_data = data_generator(
    my_data,
    test_labels,
    batch_size=1,
    shuffle=False,
)

In [ ]:
preds = model.predict(
    custom_data,
    batch_size=1,
    use_multiprocessing=True
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
preds

array([[0.9124099 , 0.08759011],
       [0.98633873, 0.01366129]], dtype=float32)

In [ ]:
# draw curves 
import matplotlib.pyplot as plt
%pylab inline
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='val')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()